In [1]:
from pyomo.environ import *

In [2]:
model = ConcreteModel()

model.j = Var(bounds=(5,9), within=NonNegativeReals) #Current density (mA/cm2)
model.C0 = Var(bounds=(200,400), within=NonNegativeReals) #Initial nickel concentration (mg/L)
model.pH0 = Var(bounds=(5,9), within=NonNegativeReals) #Initial pH of the solution (-)
model.T = Var(bounds=(30,60), within=NonNegativeReals) #Electrolysis time (min)

model.OF1 = Var(bounds=(0,100), within=NonNegativeReals) #Nickel removal efficiency (%)
model.OF2 = Var(within=NonNegativeReals) #Operating costs (US$/m3)

model.epsilon = Param(initialize=10000, mutable=True) #Definition of the ε parameter

In [3]:
model.eq1 = Constraint(expr= model.OF1== -246.32356 + 52.77326*model.j - 0.243885*model.C0 + 25.82701*model.pH0
                       + 2.31087*model.T + 0.029504*model.j*model.C0 - 1.90775*model.j*model.pH0
                       - 0.166890*model.j*model.T + 0.006918*model.C0*model.pH0 + 0.00015*model.C0*model.T               
                       - 0.035824*model.pH0*model.T - 2.22971*model.j**2 - 0.000194*model.C0**2 - 0.586381*model.pH0**2
                       - 0.005699*model.T**2) #Obtained model for the nickel removal efficiency

model.eq2 = Constraint(expr= model.OF2== -0.394510 + 0.145786*model.j - 0.000021*model.C0 - 0.004372*model.pH0
                       + 0.025888*model.T) #Obtained model for the operating costs

model.eq3 = Constraint(expr= model.OF1<=model.epsilon) #Applying the ε-constraint method

In [4]:
#Goal of each Objective Function
model.obj1 = Objective(expr=model.OF1, sense=maximize)
model.obj2 = Objective(expr=model.OF2, sense=minimize)

In [5]:
#Applying the IPOPT solver
opt = SolverFactory('ipopt')

In [6]:
#Obtain the least obliged amount of OF1
model.obj1.deactivate() 
results = opt.solve(model)

minOF1=value(model.obj1)

print('minOF1 = ',round(value(model.obj1),2))

minOF1 =  47.51


In [7]:
#Obtain the most possible amount of OF1
model.obj2.deactivate() 
model.obj1.activate() 
results = opt.solve(model)

maxOF1=value(model.obj1)

print('maxOF1 = ',round(value(model.obj1),2))

maxOF1 =  100.0


In [8]:
#Obtain the pareto front with 30 iteration
Nsteps=30
X=[]
Y=[]
print('  j    ','    C0 ','      pH0  ','    T ','       maximized OF1 ','    minimized OF2 ','    Epsilon ')
for counter in range(1,Nsteps+1):
    model.epsilon = minOF1+(maxOF1-minOF1)*(counter-1)/(Nsteps-1)
    results = opt.solve(model)
    print("%5.2f     "% value(model.j),"%5.2f"% value(model.C0),"   %5.2f"% value(model.pH0),"    %5.2f"% value(model.T),"         %5.2f"% value(model.obj1),"           %5.2f"% value(model.obj2), "           %5.2f"% value(model.epsilon))
    X.append(value(model.obj1))
    Y.append(value(model.obj2))

  j         C0        pH0       T         maximized OF1      minimized OF2      Epsilon 
 5.49      354.00     5.86     39.94          47.51             1.41            47.51
 5.54      353.55     5.92     40.11          49.32             1.42            49.32
 5.59      352.85     5.98     40.35          51.13             1.43            51.13
 5.64      351.94     6.04     40.63          52.94             1.45            52.94
 5.69      350.87     6.09     40.93          54.75             1.46            54.75
 5.74      349.65     6.14     41.25          56.56             1.48            56.56
 5.73      341.06     6.13     42.10          58.37             1.50            58.37
 5.79      340.63     6.19     42.23          60.18             1.51            60.18
 5.86      339.96     6.24     42.42          61.99             1.52            61.99
 5.92      339.11     6.30     42.63          63.80             1.54            63.80
 5.98      338.09     6.35     42.87          65.61